In [33]:
import cardapio as ptable
import log_cardapio as log
import pdfplumber

logger = log.setup_logger()

In [34]:
# Exemplo de uso
try:
    df_final = ptable.processar_pdf("cardapio_junho.pdf")
    if not df_final.empty:
        df_final.to_csv("cardapio_processado.csv", index=False)
        logger.info("Arquivo CSV salvo com sucesso!")
    else:
        logger.warning("Nenhum dado foi salvo - DataFrame vazio")
except Exception as e:
    logger.error(f"Falha no processamento do PDF: {str(e)}", exc_info=True)
    df_final = "Falha no processamento do PDF"

2025-08-06 16:45:38 (BRT) - INFO - 
############################################################
INICIANDO PROCESSAMENTO DO ARQUIVO: cardapio_junho.pdf
############################################################
2025-08-06 16:45:38 (BRT) - INFO - 
############################################################
INICIANDO PROCESSAMENTO DO ARQUIVO: cardapio_junho.pdf
############################################################
2025-08-06 16:45:38 (BRT) - INFO - 
############################################################
INICIANDO PROCESSAMENTO DO ARQUIVO: cardapio_junho.pdf
############################################################
2025-08-06 16:45:38 (BRT) - INFO - 
############################################################
INICIANDO PROCESSAMENTO DO ARQUIVO: cardapio_junho.pdf
############################################################
2025-08-06 16:45:38 (BRT) - INFO - 
############################################################
INICIANDO PROCESSAMENTO DO ARQUIVO: cardapio_junho.

In [35]:
# Exibe resultado
df_final

""


In [36]:
with pdfplumber.open("cardapio_junho.pdf") as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        print(f"\n📄 Página {i}")
        tables = page.extract_tables()
        if not tables:
            print("⚠️ Nenhuma tabela encontrada nesta página.")
            continue
        for t_idx, table in enumerate(tables):
            print(f"\nTabela {t_idx} da página {i}")
            for row in table:
                print(row)



📄 Página 1

Tabela 0 da página 1
['', 'segunda-feira, 2 de junho de', '', '', 'terça-feira, 3 de junho de', '', '', 'quarta-feira, 4 de junho', '', '', 'quinta-feira, 5 de', '', '', 'sexta-feira, 6 de junho', '']
[None, '2025', None, None, '2025', None, None, 'de 2025', None, None, 'junho de 2025', None, None, 'de 2025', None]

Tabela 1 da página 1
['Saladas', 'Acelga com tomate picado', 'Chicória com cenoura\nralada', 'Alface com rabanete', 'Almeirão com cenoura', '', 'Alface com repolho', '']
[None, None, None, None, None, None, '', None]
[None, 'Beterraba cozida', 'Chuchu colorido\nFruta', 'Grão de bico com cenoura\ncozida', 'Repolho cozido\nFruta', 'Abobrinha colorida', None, None]
[None, None, None, None, None, None, 'Abobrinha colorida', None]
[None, None, None, None, None, None, '', None]
[None, None, None, None, None, '', '', '']
['Acompanhamento', 'Arroz Branco', 'Arroz Branco', 'Arroz Branco/arroz\ncolorido', 'Arroz Branco', 'Arroz Branco', None, None]
[None, 'Feijão Preto',